<a href="https://colab.research.google.com/github/ai2ys/Python-Cheat-Sheet-As-Jupyter-Notebooks/blob/master/python_complex_enums.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to build complex enumerations objects in Python


In [1]:
!python --version

Python 3.7.12


Sample in cell below is based on code from [https://docs.python.org/3.8/library/enum.html?highlight=enum#intflag](https://docs.python.org/3.8/library/enum.html?highlight=enum#intflag)

In [2]:
from enum import auto, IntFlag
from typing import List

class Colors(IntFlag):
    RED = auto()
    BLUE = auto()
    GREEN = auto()
    MAGENTA = RED | BLUE
    CYAN = BLUE | GREEN
    WHITE = RED | BLUE | GREEN

    # def __init__(self, *args):
    #     print(args)

    def contains(self) -> List[str]:
        Colors.base_colors = [Colors.RED, Colors.GREEN, Colors.BLUE]
        return [base_color.name for base_color in Colors.base_colors if base_color in self]

for c in Colors:
    print(f"{c.name}\thas value {c.value}")        

for c in Colors:
    print(f"{c.name}\tcontains {c.contains()}")

RED	has value 1
BLUE	has value 2
GREEN	has value 4
MAGENTA	has value 3
CYAN	has value 6
WHITE	has value 7
RED	contains ['RED']
BLUE	contains ['BLUE']
GREEN	contains ['GREEN']
MAGENTA	contains ['RED', 'BLUE']
CYAN	contains ['GREEN', 'BLUE']
WHITE	contains ['RED', 'GREEN', 'BLUE']


## Defining base class for zero based auto numbering of enumerations values within class

In [3]:
from enum import Enum, auto
class AutoNumbered(Enum):
    A = auto()
    B = auto()
    C = auto()

for e in AutoNumbered:
    print(e.name, e.value)

A 1
B 2
C 3


The built-in auto numbering starts at number "1". To get an auto numbering starting at "0", the following class can be used as base class.

In [4]:
from enum import Enum

class EnumZeroBased(Enum):
    """Base class for using auto numbering with enumerations starting from zero.
    """
    def __new__(cls, *args):
        value = len(cls.__members__)
        obj = object.__new__(cls)
        obj._value_ = value
        return obj

## Sample for using the auto numbering

In [5]:
class AutoNumberedZero(EnumZeroBased):
    A = ()
    B = ()
    C = ()

for e in AutoNumberedZero:
    print(e.name, e.value)

A 0
B 1
C 2


## Sample for using the auto numbering and additional parameter for initialization

In [6]:
class CfaPattern(Enum):
    """https://www.awaresystems.be/imaging/tiff/tifftags/privateifd/exif/cfapattern.html
    https://www.awaresystems.be/imaging/tiff/tifftags/cfaplanecolor.html

    Args:
        Enum ([type]): [description]
    """
    Red = 0
    Green = 1
    Blue = 2
    Magenta = 3
    Yellow = 4
    White = 6

class BayerPixel(EnumZeroBased):
    """Enumeration defining bayer pixel.
    """   
    def __init__(self, cfa_equivalent:CfaPattern):
        """
        Args:
            rgb_equivalent: The corresponding rgb pixel color for this bayer pixel.
        """
        self._cfa_equivalent = cfa_equivalent
    
    R = (CfaPattern.Red)
    Gr = (CfaPattern.Green)
    Gb = (CfaPattern.Green)
    B = (CfaPattern.Blue)
    
    @property
    def get_cfa_equivalent(self) -> CfaPattern:
        """
        Returns:
            RGBPixel: The corresponding RGB pixel value for this bayer pixel value.
        """
        return self._cfa_equivalent


class BayerPattern(Enum):
    """Enumeration defining bayer CFA patterns, e.g. RGGB, GRBG, ...
       The start pixel defines the enum value. Therefore retrieving the pattern is possible by using
       BayerPattern(BayerPixel.value)
    """
    RGGB = (BayerPixel.R, BayerPixel.Gr, BayerPixel.Gb, BayerPixel.B)
    GRBG = (BayerPixel.Gr, BayerPixel.R, BayerPixel.B, BayerPixel.Gb)
    GBRG = (BayerPixel.Gb, BayerPixel.B, BayerPixel.R, BayerPixel.Gr)
    BGGR = (BayerPixel.B, BayerPixel.Gb, BayerPixel.Gr, BayerPixel.R)
    
    def __init__(
        self, 
        p0:CfaPattern, 
        p1:CfaPattern, 
        p2:CfaPattern, 
        p3:CfaPattern
    ):
        """
        Args:
            p0 (CfaPattern): First bayer pixel at position [0,0].
            p1 (CfaPattern): Second bayer pixel at position [0,1].
            p2 (CfaPattern): Third bayer pixel at position [1,0].
            p3 (CfaPattern): Fourth bayer pixel at position [1,1].
        """
        self._value_ = p0.value
        self._start_pixel:BayerPixel = p0
        self._list_pixel = [p0, p1, p2, p3]
        self._map_pixel_to_index = dict( (p,i) for i, p in enumerate(self._list_pixel))
    
    @property
    def start_pixel(self) -> BayerPixel:
        return self._start_pixel
    
    def get_pixel_index(self, pixel:BayerPixel) -> int:
        return self._map_pixel_to_index[pixel]


    



Sample for retrieving the startpixel

In [7]:
BayerPattern.BGGR.start_pixel

<BayerPixel.B: 3>

Printing the position of each pixel within the Bayer pattern.
Position of indices within the Bayer pattern is as following:

|||
|-|-|
|0|1|
|2|3|

In [55]:
import numpy as np
import pandas as pd

for pattern in BayerPattern:
    print(f"pattern: '{pattern.name}'\n{15*'-'}")
    array_pixel = [None]*4
    for pixel in BayerPixel:
        array_pixel[pattern.get_pixel_index(pixel)] = pixel.name
        print(f"{pixel.name:2} at index '{pattern.get_pixel_index(pixel)}'")
    df = pd.DataFrame(np.array(array_pixel).reshape(2,2))
    print(f"{15*'-'}\n{df.to_string(index=False, header=False)}\n{15*'='}")

pattern: 'RGGB'
---------------
R  at index '0'
Gr at index '1'
Gb at index '2'
B  at index '3'
---------------
  R  Gr
 Gb   B
pattern: 'GRBG'
---------------
R  at index '1'
Gr at index '0'
Gb at index '3'
B  at index '2'
---------------
 Gr   R
  B  Gb
pattern: 'GBRG'
---------------
R  at index '2'
Gr at index '3'
Gb at index '0'
B  at index '1'
---------------
 Gb   B
  R  Gr
pattern: 'BGGR'
---------------
R  at index '3'
Gr at index '2'
Gb at index '1'
B  at index '0'
---------------
  B  Gb
 Gr   R
